In [156]:
import json
import numpy as np
import pandas as pd

In [157]:
photo_datalist = []
with open('../data/photos.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        photo_datalist.append(data)

restaurant_datalist = []
with open('../data/yelp_academic_dataset_business.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        restaurant_datalist.append(data)
        


In [158]:
# clean photos
# create dataframe
picture_df = pd.DataFrame(photo_datalist)
restaurant_df = pd.DataFrame(restaurant_datalist)
michelin_df = pd.read_csv('../data/michelin.csv')
print(restaurant_df['attributes'])
print(michelin_df.columns)

0                             {'ByAppointmentOnly': 'True'}
1                    {'BusinessAcceptsCreditCards': 'True'}
2         {'BikeParking': 'True', 'BusinessAcceptsCredit...
3         {'RestaurantsDelivery': 'False', 'OutdoorSeati...
4         {'BusinessAcceptsCreditCards': 'True', 'Wheelc...
                                ...                        
150341    {'ByAppointmentOnly': 'False', 'RestaurantsPri...
150342    {'BusinessAcceptsCreditCards': 'True', 'Restau...
150343    {'RestaurantsPriceRange2': '1', 'BusinessAccep...
150344    {'BusinessParking': '{'garage': False, 'street...
150345    {'WheelchairAccessible': 'True', 'BusinessAcce...
Name: attributes, Length: 150346, dtype: object
Index(['Name', 'Address', 'Location', 'Price', 'Cuisine', 'Longitude',
       'Latitude', 'PhoneNumber', 'Url', 'WebsiteUrl', 'Award', 'GreenStar',
       'FacilitiesAndServices', 'Description'],
      dtype='object')


In [159]:
# clean michelin
michelin_df = michelin_df.drop(columns=['PhoneNumber', 'Url', 'WebsiteUrl', 'Award', 'GreenStar', 'Description', 'Location'])
# encode prices based on number of characters
michelin_df["Price"] = michelin_df["Price"].apply(lambda x: str(len(str(x))))
facilities_and_services = michelin_df["FacilitiesAndServices"].apply(lambda x: x.split(",") if isinstance(x, str) else [])
michelin_df["attributes"] = facilities_and_services.apply(lambda x: {item.strip(): True for item in x})
# Process cuisine properly - use apply with a function that has access to row index
for idx, row in michelin_df.iterrows():
    cuisine_list = row["Cuisine"].split(",") if isinstance(row["Cuisine"], str) else []
    cuisine_clean = [c.strip() for c in cuisine_list]
    price = row["Price"]
    # Update attributes for this specific row
    michelin_df.at[idx, "attributes"] = {**michelin_df.at[idx, "attributes"], 
                                         "Cuisine": cuisine_clean,
                                         "RestaurantsPriceRange2": price.strip()}
michelin_df['is_michelin'] = True
restaurant_df['is_michelin'] = False
# generate random high ratings for michelin restaurants
michelin_df['stars'] = np.random.uniform(4, 5, michelin_df.shape[0])
# generate business_ids for michelin
michelin_df['business_id'] = np.arange(len(michelin_df))

In [160]:
# align michelin_df and restaurant_df
restaurant_df = restaurant_df.drop(columns=['hours', 'review_count', 'is_open'])
michelin_df = michelin_df.drop(columns=['FacilitiesAndServices', 'Cuisine', 'Price'])
michelin_df = michelin_df.rename(columns={
    "Name": "name",
    "Address": "address",
    "Longitude": "longitude",
    "Latitude": "latitude",
})
# combine address, city, state, postal_code fields into one column
restaurant_df["address"] = restaurant_df["address"] + ", " + restaurant_df["city"] + ", " + restaurant_df["state"] + " " + restaurant_df["postal_code"]
restaurant_df = restaurant_df.drop(columns=['city', 'state', 'postal_code'])

In [161]:
non_food_terms = '|'.join(['Salon', 'Barber', 'Gym', 'Spa', 'Theater', 'Nightlife', 'Beauty', 'Barbershop', "Active Life",
    "Automotive",
    "Beauty & Spas",
    "Home Services",
    "Health & Medical",
    "Hotels & Travel",
    "Local Services",
    "Professional Services",
    "Public Services & Government",
    "Real Estate",
    "Religious Organizations",
    "Shopping & Retail",
    "Transportation",
    "Arts & Entertainment",
    "Event Planning & Services",
    "Education",
    "Financial Services",
    "Nightlife",
    "Pets & Animal Services",
    "Sports & Recreation",
    "Miscellaneous Services",
    "Shopping", "Women's Clothing", "Fashion"
])

# Filter out non-food related businesses
restaurant_df = restaurant_df[~restaurant_df['categories'].str.contains(non_food_terms, case=False, na=False)]
picture_df = picture_df[picture_df['label'] != 'inside']
picture_df = picture_df[picture_df['label'] != 'outside']
picture_df = picture_df[picture_df['label'] != 'menu']
# Keep only photos that belong to the filtered restaurants
picture_df = picture_df[picture_df['business_id'].isin(restaurant_df['business_id'])]

# Print statistics
print("Number of food establishments:", len(restaurant_df))
print("Number of photos for food establishments:", len(picture_df))
print("\nSample categories in filtered dataset:")
print(restaurant_df['categories'].unique())
print(restaurant_df.columns)

restaurant_df = restaurant_df.drop(columns=["categories"])


Number of food establishments: 51036
Number of photos for food establishments: 81941

Sample categories in filtered dataset:
['Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries'
 'Brewpubs, Breweries, Food'
 'Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants'
 ... 'Restaurants, Sandwiches, Convenience Stores, Coffee & Tea, Food'
 'Cafes, Juice Bars & Smoothies, Coffee & Tea, Restaurants, Food'
 'Specialty Food, Food, Coffee & Tea, Coffee Roasteries']
Index(['business_id', 'name', 'address', 'latitude', 'longitude', 'stars',
       'attributes', 'categories', 'is_michelin'],
      dtype='object')


In [162]:
# union michelin_df and restaurant_df based on business_id as index
michelin_df = michelin_df.set_index('business_id')
print(michelin_df["attributes"].sample(1).to_list())
restaurant_df = restaurant_df.set_index('business_id')
combined_df = pd.concat([michelin_df, restaurant_df], axis=0)
combined_df = combined_df.reset_index()
print(combined_df.columns.to_list())
print(combined_df["attributes"].sample(10).to_list())

[{'Terrace': True, 'Cuisine': ['Regional Cuisine', 'Seasonal Cuisine'], 'RestaurantsPriceRange2': '3'}]
['business_id', 'name', 'address', 'longitude', 'latitude', 'attributes', 'is_michelin', 'stars']
[{'RestaurantsPriceRange2': '1', 'RestaurantsDelivery': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'BusinessAcceptsCreditCards': 'True'}, {'Air conditioning': True, 'Counter dining': True, 'Wheelchair access': True, 'Cuisine': ['Contemporary', 'Italian'], 'RestaurantsPriceRange2': '4'}, {'Air conditioning': True, 'Car park': True, 'Cash only': True, 'Cuisine': ['Vietnamese', 'Noodles'], 'RestaurantsPriceRange2': '1'}, {'RestaurantsGoodForGroups': 'True', 'WiFi': "u'free'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}", 'RestaurantsTakeOut': 'False', 'BusinessPa

In [163]:
import pandas as pd
import ast

def extract_and_encode_attributes(df):
    """
    This function takes a dataframe with a column 'attributes' that contains nested JSON-like data.
    It flattens the nested structure, encodes list values into dummy variables, and merges the encoded
    features back into the original dataframe.

    Parameters:
      df (pd.DataFrame): Input dataframe containing at least the "attributes" column.
    
    Returns:
      pd.DataFrame: A new dataframe with encoded attributes features.
    """
    
    def flatten_dict(d, parent_key='', sep='_'):
        """
        Recursively flattens a nested dictionary, joining keys with the specified separator.
        
        Parameters:
          d (dict): The dictionary to flatten.
          parent_key (str): The string to prepend to keys (used for recursion).
          sep (str): Separator between concatenated keys.
          
        Returns:
          dict: The flattened dictionary.
        """
        items = {}
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            # If the value is a dictionary, flatten it recursively.
            if isinstance(v, dict):
                items.update(flatten_dict(v, new_key, sep=sep))
            # If the value is a string that looks like a dictionary, try to convert it.
            elif isinstance(v, str) and v.strip().startswith("{") and v.strip().endswith("}"):
                try:
                    nested = ast.literal_eval(v)
                    if isinstance(nested, dict):
                        items.update(flatten_dict(nested, new_key, sep=sep))
                    else:
                        items[new_key] = v
                except Exception:
                    items[new_key] = v
            else:
                items[new_key] = v
        return items

    def process_attributes(attr):
        """
        Processes the attribute entry from a single row.
        It converts string representations of dictionaries into a dictionary,
        and then flattens the structure.
        
        Parameters:
          attr: The attribute data (could be a dictionary, string, or None).
          
        Returns:
          dict: A flattened dictionary of attributes.
        """
        if attr is None:
            return {}
        if isinstance(attr, str):
            try:
                attr = ast.literal_eval(attr)
            except Exception:
                return {}
        if isinstance(attr, dict):
            return flatten_dict(attr)
        return {}

    # Apply processing to every row's 'attributes' column.
    flat_attributes = df['attributes'].apply(process_attributes)
    
    # Convert the list of dictionaries into a DataFrame (one row per original row).
    attr_df = pd.json_normalize(flat_attributes)
    
    # Check and encode any list values as one-hot encoded columns.
    for col in attr_df.columns:
        # If any non-null entry is a list...
        if attr_df[col].dropna().apply(lambda x: isinstance(x, list)).any():
            # Explode the list into separate rows, get dummies, and then aggregate back per row.
            dummies = attr_df[col].explode().str.get_dummies().groupby(level=0).max()
            # Rename columns with a prefix for clarity.
            dummies = dummies.add_prefix(f"{col}_")
            # Remove the original list column and join the new dummy columns.
            attr_df = attr_df.drop(columns=[col]).join(dummies)
    
    # Optionally, convert string booleans to actual booleans.
    bool_map = {'True': True, 'False': False}
    for col in attr_df.columns:
        if attr_df[col].dtype == object:
            attr_df[col] = attr_df[col].map(bool_map).fillna(attr_df[col])
    
    # Fill missing values. Adjust the fill strategy depending on your model.
    attr_df = attr_df.fillna(0)
    
    # Merge the encoded features back with the original dataframe.
    df_encoded = pd.concat([df.reset_index(drop=True), attr_df.reset_index(drop=True)], axis=1)
    return df_encoded


In [164]:
# extract attributes and create recommendation features
combined_df = extract_and_encode_attributes(combined_df)
combined_df = combined_df.drop(columns=['attributes'])

In [165]:
# check output based on column datatypes
print(combined_df.dtypes)
print(combined_df.shape)
print(combined_df.columns.to_list())

business_id           object
name                  object
address               object
longitude            float64
latitude             float64
                      ...   
Cuisine_Yakitori       int64
Cuisine_Yoshoku        int64
Cuisine_Yukhoe         int64
Cuisine_Yunnanese      int64
Cuisine_Zhejiang       int64
Length: 384, dtype: object
(68782, 384)
['business_id', 'name', 'address', 'longitude', 'latitude', 'is_michelin', 'stars', 'Air conditioning', 'Interesting wine list', 'Valet parking', 'Wheelchair access', 'RestaurantsPriceRange2', 'Garden or park', 'Restaurant offering vegetarian menus', 'Car park', 'Great view', 'Terrace', 'Counter dining', 'Notable sake list', 'Shoes must be removed', 'Cash only', 'Brunch', 'Credit cards not accepted', 'Bring your own bottle', 'Cash only - lunch', 'Foreign credit cards not accepted', 'Booking essential', 'Booking essential - dinner', 'RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking_garage', 'Busin

In [166]:
# New cell for validation
# Check if all pictures link to valid restaurants
# get business_ids of non-michelin restaurants
valid_business_ids = set(combined_df[~combined_df['is_michelin']]['business_id'])
invalid_pictures = picture_df[~picture_df['business_id'].isin(valid_business_ids)]

print("Validation Results:")
print(f"Total pictures: {len(picture_df)}")
print(f"Pictures with valid restaurant links: {len(picture_df) - len(invalid_pictures)}")
print(f"Pictures with invalid restaurant links: {len(invalid_pictures)}")

if len(invalid_pictures) > 0:
    print("\nSample of invalid picture entries:")
    print(invalid_pictures.head())
else:
    print("\nAll pictures are linked to valid restaurants!")

# Optional: Remove any invalid pictures if found
if len(invalid_pictures) > 0:
    picture_df = picture_df[picture_df['business_id'].isin(valid_business_ids)]
    print(f"\nCleaned dataset now contains {len(picture_df)} valid pictures")




Validation Results:
Total pictures: 81941
Pictures with valid restaurant links: 81941
Pictures with invalid restaurant links: 0

All pictures are linked to valid restaurants!


In [167]:
# parse photos folder and remove any photos that are not in the cleaned dataset
import os

# Create a set of valid photo IDs
valid_photo_ids = set(picture_df['photo_id'])

# Define the path to the photos folder
photos_folder = '../data/photos'

# Iterate over all files in the photos folder
for filename in os.listdir(photos_folder):
    # Extract the photo ID from the filename
    photo_id = filename.split('.')[0]
    
    # Check if the photo ID is not in the valid set
    if photo_id not in valid_photo_ids:
        # Construct the full path to the file
        file_path = os.path.join(photos_folder, filename)
        
        # Remove the file
        os.remove(file_path)
        print(f"Removed invalid photo: {filename}")
        
# Check the number of photos in the folder
print(f"Remaining photos in the folder: {len(os.listdir(photos_folder))}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/photos'

In [168]:
# save restaurants to json for db import and csv for model training
restaurant_df_model_training = combined_df.copy().drop(columns=['name', 'address'])
picture_df.to_csv('../data/cleaned_photos.csv', index=False)

# keep name, address, stars, latitude, longitude, is_michelin, business_id, RestaurantsPriceRange2
# Select important columns for database
columns_to_keep = ['name', 'address', 'stars', 'latitude', 'longitude', 'is_michelin', 'business_id']

# Find the RestaurantsPriceRange2 column (it might be extracted from attributes)
price_col = [col for col in combined_df.columns if 'RestaurantsPriceRange2' in col]

# convert price_col to string
combined_df[price_col] = combined_df[price_col].astype(str)

# Create database version with selected columns
restaurant_df_database = combined_df[columns_to_keep + price_col].copy()
picture_df.to_json('../data/cleaned_photos.json', orient='records', lines=True)

restaurant_df_database.to_json('../data/cleaned_restaurants.json', orient='records', lines=True)
restaurant_df_model_training.to_csv('../data/cleaned_restaurants.csv', index=False)